<a href="https://colab.research.google.com/github/san-258/10ema5-10cross-Oct1st/blob/main/10ema5_10cross_Oct1st.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install alpaca-trade-api pandas numpy python-dotenv requests pytz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.0/725.0 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 9.2 MB/s eta 0:00:00
  Created wheel for msgpack: filename=msgpack-1.0.3-cp312-cp312-linux_x86_64.whl size=15688 sha256=faa2cc293bae0a89eb8479761c48512dd1b5bc0e272fd7cf8764e8ca1d237e86
  Stored in directory: /root/.cache/pip/wheels/ba/bd/3f/f043e8f634db9c90ae128d631f43ae9990eef01274a63291f9
  Created wheel for websockets: filename=websockets-10.4-cp312-cp312-linux_x86_64.whl size=107329 sha256=42c091193976a47840ef13ecfe73beec50d86c28c5b7285ded95c37434b949fa
  Stored in directory: /root/.cache/pip/wheels/80/cf/6d/5d7e4c920cb41925a17

In [2]:
ALPACA_API_KEY = "PKEG6FEXHET3ACX1MI7S"
ALPACA_SECRET_KEY = "c5IN71n5vPOwBe3hmxShPvUfWf0DjwLsNbSb8FN4"
ALPACA_BASE_URL = "https://paper-api.alpaca.markets"
DISCORD_WEBHOOK_URL = "https://discord.com/api/webhooks/1414722189529448551/xt8TBox1-w7WQ9ZMW9iCW4twLM7UtBXFpZJ96-KmAFtJ926C9z8L2IIyI3SOJCQbDh9i"

In [ ]:
# -*- coding: utf-8 -*-
"""Enhanced 5-10 EMA Cross Trading Bot - Google Colab Optimized"""

# First, install required packages
print("Installing required packages...")
import subprocess
import sys

packages = [
    'alpaca-trade-api',
    'pandas',
    'numpy',
    'python-dotenv',
    'requests',
    'pytz',
    'nest-asyncio'
]

for package in packages:
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])
    except:
        print(f"Warning: Could not install {package}")

print("✓ Packages installed\n")

import os
import pandas as pd
import numpy as np
import asyncio
import logging
import time
import threading
from datetime import datetime, timedelta
from collections import deque
from typing import Dict, Optional
import warnings
import requests
import pytz
import nest_asyncio

# Apply nest_asyncio for Colab compatibility
nest_asyncio.apply()

warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION - PUT YOUR CREDENTIALS HERE
# ============================================================================
ALPACA_API_KEY = "PKEG6FEXHET3ACX1MI7S"
ALPACA_SECRET_KEY = "c5IN71n5vPOwBe3hmxShPvUfWf0DjwLsNbSb8FN4"
ALPACA_BASE_URL = "https://paper-api.alpaca.markets"  # Use paper trading
# For live trading, change to: "https://api.alpaca.markets"

DISCORD_WEBHOOK_URL = "YOUR_DISCORD_WEBHOOK_URL_HERE"  # Optional
# ============================================================================

print(f"✓ API Key configured: {bool(ALPACA_API_KEY and ALPACA_API_KEY != 'YOUR_ALPACA_API_KEY_HERE')}")
print(f"✓ Discord configured: {bool(DISCORD_WEBHOOK_URL and DISCORD_WEBHOOK_URL != 'YOUR_DISCORD_WEBHOOK_URL_HERE')}\n")

# Import Alpaca
try:
    import alpaca_trade_api as tradeapi
    from alpaca_trade_api.stream import Stream
    ALPACA_AVAILABLE = True
    print("✓ Alpaca Trade API loaded")
except ImportError as e:
    print(f"✗ Alpaca Trade API import failed: {e}")
    ALPACA_AVAILABLE = False

# Logging setup (Colab-friendly)
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger(__name__)


class LiveTradingBot:
    def __init__(self, initial_capital=250000, risk_per_trade=0.01):
        """Initialize the enhanced trading bot"""

        # Capital & Risk Management
        self.initial_capital = initial_capital
        self.risk_per_trade = risk_per_trade
        self.max_position_size = initial_capital * 0.15

        # Exit Parameters
        self.stop_loss = 0.008  # 0.8%
        self.take_profit_1 = 0.015  # 1.5%
        self.take_profit_2 = 0.025  # 2.5%
        self.trailing_stop = 0.005  # 0.5%

        # Signal Configuration
        self.signal_threshold = 4  # Lowered from 6 for more activity

        # Data Storage
        self.symbol_data = {}
        self.max_bars = 200
        self.positions = {}
        self.pending_orders = {}

        # Symbols to trade
        self.symbols = ['GOOGL', 'PDD', 'MSFT', 'APP', 'PEP', 'AVGO', 'RDDT', 'CAT', 'LMT']

        # Status tracking
        self.bot_start_time = datetime.now()
        self.last_activity_time = datetime.now()
        self.total_bars_received = 0
        self.total_signals_generated = 0
        self.total_trades_executed = 0
        self.is_running = False
        self.status_thread = None

        # Statistics
        self.stats = {
            'bars_per_symbol': {},
            'signals_per_symbol': {},
            'trades_per_symbol': {},
            'last_price_per_symbol': {},
            'connection_status': 'Disconnected',
            'sl_exits': 0,
            'pt_exits': 0,
            'after_hours_bars': 0,
            'market_hours_bars': 0
        }

        # Initialize tracking for each symbol
        for symbol in self.symbols:
            self.symbol_data[symbol] = deque(maxlen=self.max_bars)
            self.positions[symbol] = {
                'qty': 0,
                'entry_price': 0,
                'entry_time': None,
                'highest_price': 0,
                'type': None
            }
            self.stats['bars_per_symbol'][symbol] = 0
            self.stats['signals_per_symbol'][symbol] = 0
            self.stats['trades_per_symbol'][symbol] = 0
            self.stats['last_price_per_symbol'][symbol] = 0.0

        # Setup connections
        self.api = None
        self.stream = None
        self.setup_alpaca_connection()

    def is_market_hours(self) -> bool:
        """Check if current time is during market hours (9:30 AM - 4:00 PM ET)"""
        try:
            eastern = pytz.timezone('US/Eastern')
            now_et = datetime.now(eastern)

            # Check if weekend
            if now_et.weekday() >= 5:  # Saturday=5, Sunday=6
                return False

            # Market hours: 9:30 AM - 4:00 PM ET
            market_open = now_et.replace(hour=9, minute=30, second=0, microsecond=0)
            market_close = now_et.replace(hour=16, minute=0, second=0, microsecond=0)

            return market_open <= now_et <= market_close

        except Exception as e:
            logger.error(f"Error checking market hours: {e}")
            return True

    def setup_alpaca_connection(self):
        """Setup connection to Alpaca API"""
        if not ALPACA_AVAILABLE:
            logger.error("Alpaca API not available")
            return

        try:
            if not ALPACA_API_KEY or ALPACA_API_KEY == "YOUR_ALPACA_API_KEY_HERE":
                logger.error("❌ Missing Alpaca credentials - please update ALPACA_API_KEY in the code")
                return

            self.api = tradeapi.REST(
                key_id=ALPACA_API_KEY,
                secret_key=ALPACA_SECRET_KEY,
                base_url=ALPACA_BASE_URL
            )

            # Test connection
            account = self.api.get_account()
            logger.info(f"✓ Connected to Alpaca - Account Status: {account.status}")

            # Setup streaming
            self.stream = Stream(
                ALPACA_API_KEY,
                ALPACA_SECRET_KEY,
                base_url=ALPACA_BASE_URL,
                data_feed='iex'
            )

        except Exception as e:
            logger.error(f"Failed to connect to Alpaca: {e}")
            self.api = None
            self.stream = None

    def calculate_indicators(self, symbol: str) -> Optional[Dict]:
        """Calculate technical indicators for a symbol"""
        if symbol not in self.symbol_data or len(self.symbol_data[symbol]) < 50:
            return None

        bars_list = list(self.symbol_data[symbol])
        df = pd.DataFrame(bars_list)

        if df.empty or len(df) < 50:
            return None

        try:
            # EMAs
            df['EMA_5'] = df['close'].ewm(span=5, adjust=False).mean()
            df['EMA_10'] = df['close'].ewm(span=10, adjust=False).mean()
            df['EMA_20'] = df['close'].ewm(span=20, adjust=False).mean()
            df['EMA_50'] = df['close'].ewm(span=50, adjust=False).mean()

            # MACD
            exp1 = df['close'].ewm(span=12, adjust=False).mean()
            exp2 = df['close'].ewm(span=26, adjust=False).mean()
            df['MACD'] = exp1 - exp2
            df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
            df['MACD_histogram'] = df['MACD'] - df['MACD_signal']

            # RSI
            delta = df['close'].diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
            rs = gain / loss
            df['RSI'] = 100 - (100 / (1 + rs))

            # ATR
            df['high_low'] = df['high'] - df['low']
            df['high_close'] = abs(df['high'] - df['close'].shift())
            df['low_close'] = abs(df['low'] - df['close'].shift())
            df['tr'] = df[['high_low', 'high_close', 'low_close']].max(axis=1)
            df['ATR'] = df['tr'].rolling(window=14).mean()

            # Volume
            df['volume_sma'] = df['volume'].rolling(window=20).mean()
            df['volume_ratio'] = df['volume'] / df['volume_sma']

            # Get latest and previous values
            latest = df.iloc[-1]
            prev = df.iloc[-2] if len(df) > 1 else latest

            return {
                'close': latest['close'],
                'EMA_5': latest['EMA_5'],
                'EMA_10': latest['EMA_10'],
                'EMA_20': latest['EMA_20'],
                'EMA_50': latest['EMA_50'],
                'MACD': latest['MACD'],
                'MACD_signal': latest['MACD_signal'],
                'MACD_histogram': latest['MACD_histogram'],
                'RSI': latest['RSI'],
                'ATR': latest['ATR'],
                'volume_ratio': latest['volume_ratio'],
                'prev_EMA_5': prev['EMA_5'],
                'prev_EMA_10': prev['EMA_10'],
                'prev_MACD': prev['MACD'],
                'prev_MACD_signal': prev['MACD_signal']
            }

        except Exception as e:
            logger.error(f"Error calculating indicators for {symbol}: {e}")
            return None

    def generate_signal(self, symbol: str, indicators: Dict) -> int:
        """Generate trading signal based on indicators"""
        try:
            # Crossover detection
            ema_5_cross_above = (
                indicators['EMA_5'] > indicators['EMA_10'] and
                indicators['prev_EMA_5'] <= indicators['prev_EMA_10']
            )
            ema_5_cross_below = (
                indicators['EMA_5'] < indicators['EMA_10'] and
                indicators['prev_EMA_5'] >= indicators['prev_EMA_10']
            )

            # MACD conditions
            macd_bullish = indicators['MACD'] > indicators['MACD_signal']
            macd_bearish = indicators['MACD'] < indicators['MACD_signal']

            # Price position
            price_above_20ema = indicators['close'] > indicators['EMA_20']
            price_below_20ema = indicators['close'] < indicators['EMA_20']

            # RSI filter
            rsi_neutral = 30 < indicators['RSI'] < 70

            # Volume confirmation
            volume_above_avg = indicators['volume_ratio'] > 1.1

            # Trend strength
            strong_uptrend = (
                indicators['EMA_5'] > indicators['EMA_10'] >
                indicators['EMA_20'] > indicators['EMA_50']
            )
            strong_downtrend = (
                indicators['EMA_5'] < indicators['EMA_10'] <
                indicators['EMA_20'] < indicators['EMA_50']
            )

            # Calculate scores
            buy_score = 0
            sell_score = 0

            # Buy signals
            if ema_5_cross_above:
                buy_score += 3
            if macd_bullish:
                buy_score += 2
            if price_above_20ema:
                buy_score += 1
            if strong_uptrend:
                buy_score += 2
            if rsi_neutral:
                buy_score += 1
            if volume_above_avg:
                buy_score += 1

            # Sell signals
            if ema_5_cross_below:
                sell_score += 3
            if macd_bearish:
                sell_score += 2
            if price_below_20ema:
                sell_score += 1
            if strong_downtrend:
                sell_score += 2
            if rsi_neutral:
                sell_score += 1
            if volume_above_avg:
                sell_score += 1

            # Track signal generation
            if buy_score >= self.signal_threshold or sell_score >= self.signal_threshold:
                self.total_signals_generated += 1
                self.stats['signals_per_symbol'][symbol] += 1

            # Return signal
            if buy_score >= self.signal_threshold:
                logger.info(f"🟢 BUY SIGNAL: {symbol} | Score: {buy_score}/{self.signal_threshold} | "
                          f"RSI: {indicators['RSI']:.1f} | Price: ${indicators['close']:.2f}")
                return 1
            elif sell_score >= self.signal_threshold:
                logger.info(f"🔴 SELL SIGNAL: {symbol} | Score: {sell_score}/{self.signal_threshold} | "
                          f"RSI: {indicators['RSI']:.1f} | Price: ${indicators['close']:.2f}")
                return -1
            else:
                return 0

        except Exception as e:
            logger.error(f"Error generating signal for {symbol}: {e}")
            return 0

    def calculate_position_size(self, symbol: str, current_price: float, atr: float) -> int:
        """Calculate position size based on risk management"""
        try:
            account = self.api.get_account()
            buying_power = float(account.buying_power)

            risk_amount = min(
                buying_power * self.risk_per_trade,
                self.max_position_size * self.risk_per_trade
            )

            stop_distance = max(atr * 2, current_price * self.stop_loss)
            position_size = int(risk_amount / stop_distance)

            max_shares = int(self.max_position_size / current_price)
            position_size = min(position_size, max_shares)

            max_affordable = int(buying_power * 0.95 / current_price)
            position_size = min(position_size, max_affordable)

            return max(position_size, 0)

        except Exception as e:
            logger.error(f"Error calculating position size for {symbol}: {e}")
            return 0

    def place_order(self, symbol: str, qty: int, side: str, current_price: float) -> Optional[str]:
        """Place order with Alpaca"""
        try:
            if qty <= 0:
                logger.warning(f"Invalid quantity for {symbol}: {qty}")
                return None

            order = self.api.submit_order(
                symbol=symbol,
                qty=qty,
                side=side,
                type='market',
                time_in_force='day'
            )

            logger.info(f"📤 ORDER PLACED: {side.upper()} {qty} shares of {symbol} at ~${current_price:.2f}")

            self.total_trades_executed += 1
            self.stats['trades_per_symbol'][symbol] += 1

            self.pending_orders[symbol] = {
                'order_id': order.id,
                'side': side,
                'qty': qty,
                'price': current_price,
                'timestamp': datetime.now()
            }

            self.send_discord(f"📊 **{side.upper()}** {qty} shares of **{symbol}** @ ${current_price:.2f}")

            return order.id

        except Exception as e:
            logger.error(f"Error placing order for {symbol}: {e}")
            return None

    def send_discord(self, message: str):
        """Send message to Discord webhook"""
        if not DISCORD_WEBHOOK_URL or DISCORD_WEBHOOK_URL == "YOUR_DISCORD_WEBHOOK_URL_HERE":
            return

        try:
            requests.post(DISCORD_WEBHOOK_URL, json={"content": message}, timeout=10)
        except:
            pass

    def check_exit_conditions(self, symbol: str, current_price: float) -> Optional[str]:
        """Check if position should be exited (SL/PT/Trailing)"""
        position = self.positions[symbol]

        if position['qty'] == 0:
            return None

        try:
            if position['type'] == 'LONG':
                if current_price > position['highest_price']:
                    position['highest_price'] = current_price

                pnl_pct = (current_price - position['entry_price']) / position['entry_price']

                if pnl_pct <= -self.stop_loss:
                    self.stats['sl_exits'] += 1
                    return "Stop Loss"

                if pnl_pct >= self.take_profit_2:
                    self.stats['pt_exits'] += 1
                    return "Take Profit 2"

                if pnl_pct >= self.take_profit_1:
                    self.stats['pt_exits'] += 1
                    return "Take Profit 1"

                if pnl_pct > 0.005:
                    if current_price < position['highest_price'] * (1 - self.trailing_stop):
                        return "Trailing Stop"

            return None

        except Exception as e:
            logger.error(f"Error checking exit conditions for {symbol}: {e}")
            return None

    def update_position(self, symbol: str, qty: int, price: float, side: str):
        """Update internal position tracking"""
        position = self.positions[symbol]

        if side == 'buy':
            position['qty'] = qty
            position['entry_price'] = price
            position['entry_time'] = datetime.now()
            position['highest_price'] = price
            position['type'] = 'LONG'

            logger.info(f"✅ LONG POSITION OPENED: {qty} shares of {symbol} at ${price:.2f}")

        elif side == 'sell':
            if position['qty'] > 0:
                pnl = position['qty'] * (price - position['entry_price'])
                pnl_pct = (price - position['entry_price']) / position['entry_price'] * 100

                logger.info(f"✅ POSITION CLOSED: {symbol} | P&L: ${pnl:.2f} ({pnl_pct:+.2f}%)")
                self.send_discord(f"💰 **CLOSED** {symbol} | P&L: ${pnl:.2f} ({pnl_pct:+.2f}%)")

            position['qty'] = 0
            position['entry_price'] = 0
            position['entry_time'] = None
            position['highest_price'] = 0
            position['type'] = None

    async def bar_callback(self, bar):
        """Handle incoming bar data"""
        try:
            symbol = bar.symbol

            self.last_activity_time = datetime.now()
            self.total_bars_received += 1
            self.stats['bars_per_symbol'][symbol] += 1
            self.stats['last_price_per_symbol'][symbol] = bar.close

            if self.is_market_hours():
                self.stats['market_hours_bars'] += 1
            else:
                self.stats['after_hours_bars'] += 1

            bar_data = {
                'timestamp': bar.timestamp,
                'open': bar.open,
                'high': bar.high,
                'low': bar.low,
                'close': bar.close,
                'volume': bar.volume
            }
            self.symbol_data[symbol].append(bar_data)

            indicators = self.calculate_indicators(symbol)
            if not indicators:
                return

            current_price = bar.close

            # Check exit conditions (24/7)
            exit_reason = self.check_exit_conditions(symbol, current_price)
            if exit_reason and self.positions[symbol]['qty'] != 0:
                logger.info(f"🚪 EXIT: {symbol} - {exit_reason} | ${current_price:.2f}")

                qty = abs(self.positions[symbol]['qty'])
                order_id = self.place_order(symbol, qty, 'sell', current_price)

                if order_id:
                    self.update_position(symbol, 0, current_price, 'sell')
                return

            # Entry signals (only during market hours)
            if self.positions[symbol]['qty'] == 0 and self.is_market_hours():
                signal = self.generate_signal(symbol, indicators)

                if signal == 1:  # BUY
                    qty = self.calculate_position_size(symbol, current_price, indicators['ATR'])

                    if qty > 0:
                        order_id = self.place_order(symbol, qty, 'buy', current_price)
                        if order_id:
                            self.update_position(symbol, qty, current_price, 'buy')

        except Exception as e:
            logger.error(f"Error in bar callback for {bar.symbol}: {e}")

    def print_status(self):
        """Print status update (Colab-friendly)"""
        from IPython.display import clear_output

        while self.is_running:
            try:
                clear_output(wait=True)

                current_time = datetime.now()
                uptime = current_time - self.bot_start_time

                market_open = self.is_market_hours()
                market_status = "🟢 OPEN" if market_open else "🔴 CLOSED"

                try:
                    account = self.api.get_account()
                    portfolio_value = float(account.portfolio_value)
                    buying_power = float(account.buying_power)
                except:
                    portfolio_value = 0
                    buying_power = 0

                active_positions = sum(1 for pos in self.positions.values() if pos['qty'] != 0)

                print("=" * 80)
                print(f"🤖 ENHANCED EMA TRADING BOT - GOOGLE COLAB")
                print("=" * 80)
                print(f"Time: {current_time.strftime('%Y-%m-%d %H:%M:%S')} | Market: {market_status}")
                print(f"Uptime: {str(uptime).split('.')[0]} | Connection: {self.stats['connection_status']}")
                print()
                print(f"Portfolio: ${portfolio_value:,.2f} | Buying Power: ${buying_power:,.2f}")
                print(f"Active Positions: {active_positions}")
                print()
                print(f"Bars: {self.total_bars_received:,} | Signals: {self.total_signals_generated} | Trades: {self.total_trades_executed}")
                print(f"SL Exits: {self.stats['sl_exits']} | PT Exits: {self.stats['pt_exits']}")
                print()
                print("SYMBOLS:")
                for symbol in self.symbols:
                    price = self.stats['last_price_per_symbol'][symbol]
                    position = self.positions[symbol]

                    if position['qty'] != 0:
                        pnl_pct = (price - position['entry_price']) / position['entry_price'] * 100
                        print(f"🟢 {symbol}: ${price:.2f} | {position['qty']} shares ({pnl_pct:+.1f}%)")
                    elif price > 0:
                        print(f"⚪ {symbol}: ${price:.2f}")

                print("=" * 80)
                print(f"Threshold: {self.signal_threshold} | Market Hours Filter: ON | SL/PT: 24/7")
                print("To stop: Interrupt the kernel (Runtime > Interrupt execution)")
                print("=" * 80)

                time.sleep(10)  # Update every 10 seconds in Colab

            except Exception as e:
                logger.error(f"Error in status display: {e}")
                time.sleep(10)

    def start_status_monitor(self):
        """Start the status monitoring thread"""
        self.status_thread = threading.Thread(target=self.print_status, daemon=True)
        self.status_thread.start()

    def start_streaming(self):
        """Start live data streaming (Colab-optimized)"""
        if not self.stream:
            logger.error("Stream not initialized")
            return

        self.is_running = True
        self.start_status_monitor()

        try:
            for symbol in self.symbols:
                self.stream.subscribe_bars(self.bar_callback, symbol)

            logger.info(f"🚀 Streaming started for {len(self.symbols)} symbols")
            logger.info(f"   Running in Google Colab")
            logger.info(f"   Signal Threshold: {self.signal_threshold}")

            self.stats['connection_status'] = 'Streaming'
            self.stream.run()

        except KeyboardInterrupt:
            self.is_running = False
            logger.info("\n🛑 Bot stopped by user")
        except Exception as e:
            logger.error(f"Stream error: {e}")
            self.stats['connection_status'] = 'Error'

    def get_account_summary(self):
        """Display account summary"""
        try:
            account = self.api.get_account()
            positions = self.api.list_positions()

            print("\n" + "=" * 80)
            print("📊 ACCOUNT SUMMARY")
            print("=" * 80)
            print(f"Portfolio Value:  ${float(account.portfolio_value):,.2f}")
            print(f"Buying Power:     ${float(account.buying_power):,.2f}")
            print(f"Cash:             ${float(account.cash):,.2f}")
            print(f"Market Status:    {'🟢 OPEN' if self.is_market_hours() else '🔴 CLOSED'}")
            print()
            print(f"Signal Threshold: {self.signal_threshold} (lowered from 6)")
            print(f"Stop Loss:        {self.stop_loss*100:.1f}%")
            print(f"Take Profit 1:    {self.take_profit_1*100:.1f}%")
            print(f"Take Profit 2:    {self.take_profit_2*100:.1f}%")

            if positions:
                print(f"\nCURRENT POSITIONS ({len(positions)})")
                print("-" * 80)

                for pos in positions:
                    unrealized_pl = float(pos.unrealized_pl)
                    unrealized_plpc = float(pos.unrealized_plpc) * 100
                    entry_price = float(pos.avg_entry_price)
                    current_price = float(pos.current_price)

                    print(f"{pos.symbol}: {pos.qty} shares @ ${entry_price:.2f}")
                    print(f"  Current: ${current_price:.2f} | P&L: ${unrealized_pl:+.2f} ({unrealized_plpc:+.2f}%)")

                    sl_price = entry_price * (1 - self.stop_loss)
                    pt1_price = entry_price * (1 + self.take_profit_1)
                    pt2_price = entry_price * (1 + self.take_profit_2)

                    print(f"  Levels: SL=${sl_price:.2f} | PT1=${pt1_price:.2f} | PT2=${pt2_price:.2f}")
            else:
                print("\nNo current positions")

            print("=" * 80)

        except Exception as e:
            logger.error(f"Error getting account summary: {e}")

    def sync_positions(self):
        """Sync with existing Alpaca positions"""
        try:
            positions = self.api.list_positions()

            if positions:
                print("\n📊 SYNCING EXISTING POSITIONS")
                print("-" * 80)

            for pos in positions:
                symbol = pos.symbol
                qty = int(float(pos.qty))
                avg_entry = float(pos.avg_entry_price)
                current_price = float(pos.current_price)

                if symbol in self.positions:
                    self.positions[symbol]['qty'] = qty
                    self.positions[symbol]['entry_price'] = avg_entry
                    self.positions[symbol]['entry_time'] = datetime.now()
                    self.positions[symbol]['highest_price'] = current_price
                    self.positions[symbol]['type'] = 'LONG' if qty > 0 else None

                    print(f"✓ {symbol}: {qty} shares @ ${avg_entry:.2f}")

            if positions:
                print("-" * 80)

        except Exception as e:
            logger.error(f"Error syncing positions: {e}")


def run_colab_bot():
    """Main function for Google Colab"""
    print("\n" + "=" * 80)
    print("🤖 ENHANCED EMA TRADING BOT - GOOGLE COLAB")
    print("=" * 80)
    print()

    # Initialize bot
    bot = LiveTradingBot(initial_capital=250000, risk_per_trade=0.01)

    if not bot.api:
        print("❌ Cannot start bot - Alpaca connection failed")
        print("   Please update ALPACA_API_KEY and ALPACA_SECRET_KEY in the code")
        return

    # Send startup notification
    bot.send_discord(
        f"🚀 **Enhanced Bot Starting (Colab)**\n"
        f"• Threshold: {bot.signal_threshold}\n"
        f"• Market Hours Filter: ON\n"
        f"• SL/PT: Active 24/7"
    )

    # Sync existing positions
    bot.sync_positions()

    # Determine trading mode
    is_paper = "paper-api" in ALPACA_BASE_URL.lower()
    mode = "📝 PAPER TRADING" if is_paper else "🔴 LIVE TRADING"

    print(f"\nACCOUNT TYPE: {mode}")
    print(f"API Endpoint: {ALPACA_BASE_URL}")
    print(f"Market Status: {'🟢 OPEN' if bot.is_market_hours() else '🔴 CLOSED'}")

    # Display account summary
    bot.get_account_summary()

    print("\n" + "=" * 80)
    print("⚡ KEY ENHANCEMENTS")
    print("=" * 80)
    print("✓ Signal threshold: 4 (lowered from 6 for more activity)")
    print("✓ Market hours filter: Entries only 9:30 AM - 4:00 PM ET")
    print("✓ Stop Loss exits: Active 24/7 (closes stuck positions)")
    print("✓ Profit Target exits: Active 24/7")
    print("✓ Enhanced monitoring with real-time P&L")
    print("✓ Google Colab optimized with nest_asyncio")
    print("✓ All 9 original symbols")
    print("=" * 80)

    # Confirmation
    print("\n🎯 SELECT MODE:")
    print("1. Start Trading Bot")
    print("2. View Account Summary Only")
    print("3. Exit")

    try:
        choice = input("\nEnter choice (1-3): ").strip()

        if choice == '1':
            if is_paper:
                confirm = input("\n📝 Start paper trading bot? (yes/no): ").strip().lower()
                if confirm == 'yes':
                    print("\n" + "=" * 80)
                    print("🚀 STARTING ENHANCED PAPER TRADING BOT")
                    print("=" * 80)
                    print("✓ Running in Google Colab")
                    print("✓ Real-time status will update every 10 seconds")
                    print("✓ To stop: Runtime > Interrupt execution")
                    print("=" * 80)
                    print("\nInitializing streaming...")

                    time.sleep(2)
                    bot.start_streaming()
                else:
                    print("❌ Paper trading cancelled")
            else:
                print("\n⚠️  WARNING: LIVE TRADING WITH REAL MONEY ⚠️")
                confirm = input("\nType 'YES' to confirm live trading: ").strip()
                double_confirm = input("Type 'TRADE' to proceed: ").strip()

                if confirm == 'YES' and double_confirm == 'TRADE':
                    print("\n" + "=" * 80)
                    print("🔴 STARTING LIVE TRADING BOT")
                    print("=" * 80)
                    print("⚠️  TRADING WITH REAL MONEY!")
                    print("✓ Running in Google Colab")
                    print("✓ Real-time status will update every 10 seconds")
                    print("✓ To stop: Runtime > Interrupt execution")
                    print("=" * 80)
                    print("\nInitializing streaming...")

                    time.sleep(2)
                    bot.start_streaming()
                else:
                    print("❌ Live trading cancelled - good choice for safety!")

        elif choice == '2':
            bot.get_account_summary()

        elif choice == '3':
            print("👋 Exiting...")

        else:
            print("❌ Invalid choice")

    except KeyboardInterrupt:
        print("\n\n🛑 Bot stopped by user")
    except Exception as e:
        logger.error(f"Bot error: {e}")
        bot.send_discord(f"❌ **Bot Error:** {str(e)}")


# Run the bot when the cell is executed
if __name__ == "__main__":
    print("=" * 80)
    print("🚀 ENHANCED EMA TRADING BOT FOR GOOGLE COLAB")
    print("=" * 80)
    print("\n⚠️  IMPORTANT: Update your API credentials in the code above!")
    print("Look for the CONFIGURATION section and replace:")
    print("  • ALPACA_API_KEY")
    print("  • ALPACA_SECRET_KEY")
    print("  • DISCORD_WEBHOOK_URL (optional)")
    print("\nThen run this cell to start the bot.")
    print("=" * 80)
    print("\n")

    # Check if credentials are configured
    if ALPACA_API_KEY != "YOUR_ALPACA_API_KEY_HERE":
        run_colab_bot()
    else:
        print("⚠️  Please configure your API credentials first!")
        print("\n📝 Quick Setup:")
        print("1. Get your Alpaca API keys from: https://alpaca.markets")
        print("2. Update the CONFIGURATION section at the top of this code")
        print("3. Run this cell again")
        print("\n💡 Tip: Use Paper Trading API for testing (already set by default)")

🤖 ENHANCED EMA TRADING BOT - GOOGLE COLAB
Time: 2025-10-03 15:59:09 | Market: 🟢 OPEN
Uptime: 1:38:44 | Connection: Streaming

Portfolio: $282,943.90 | Buying Power: $577,668.62
Active Positions: 8

Bars: 646 | Signals: 10 | Trades: 6
SL Exits: 1 | PT Exits: 1

SYMBOLS:
🟢 GOOGL: $244.77 | 153 shares (+0.4%)
🟢 PDD: $133.03 | 280 shares (-0.4%)
🟢 MSFT: $519.77 | 73 shares (+1.3%)
🟢 APP: $685.04 | 55 shares (+0.9%)
🟢 PEP: $142.93 | 263 shares (+0.3%)
🟢 AVGO: $340.56 | 110 shares (+0.2%)
🟢 RDDT: $210.78 | 178 shares (+0.2%)
🟢 CAT: $503.16 | 77 shares (+3.8%)
⚪ LMT: $505.85
Threshold: 4 | Market Hours Filter: ON | SL/PT: 24/7
To stop: Interrupt the kernel (Runtime > Interrupt execution)
